# costPower

In [27]:
import requests
import pymongo
res=requests.get("https://sip.einvoice.nat.gov.tw/ods-main/ODS308E/download/691C0280-CEFB-488F-9E71-6AA4F39A41CD/8/1124193D-09F5-4711-AB9A-01848E3B88E4/8/?fileType=csv")
x=res.text
keys=x.replace('"',"").replace("\u3000","").replace("\U000fffb4","").replace("\r","").split("\n")[0].split(",")
values=[data.replace("\u3000","")
        .replace("\U000fffb4","")
        .replace("\U000fb56f","塭")
        .replace("\r","")
        .split(",") for data in x.replace('"',"").split("\n")[1:]]
datalist=[]
for valueList in values:
    data={}
    for key,value in zip(keys,valueList):
        data[key]=value
    datalist.append(data)
dataFinal=[]
for i in datalist:
    data={}
    if i.get('主行業別',"")=='餐飲業' and i.get('年度',"")=='2017' and i.get("消費熱度計算來源","")=="電子發票":
        data['add']=i['縣市']+i['鄉鎮市區']+i['村里']
        data['bigadd']=i['縣市'].replace("臺","台")
        data['smalladd']=i['鄉鎮市區']
        data['costPower']=i['銷售額指標']
        dataFinal.append(data)
        
client=pymongo.mongo_client.MongoClient("localhost",27017,username='j122085',password='850605')
# print(client.database_names())
db=client.rawData
print(db.collection_names())
collection=db.addressCoordinate
Coordinate={add['add']:{"lat":add['lat'],'lng':add['lng']} for add in list(collection.find({}))}
client.close()

for add in dataFinal:
    try:
        add['lat']=Coordinate[add['add']]['lat']
        add['lng']=Coordinate[add['add']]['lng']
        add["_id"]=add.pop('add')
    except:
#         print(add['add'])
        pass
costPower=[add for add in dataFinal if 'lat' in add]
client=pymongo.mongo_client.MongoClient("localhost",27017,username='j122085',password='850605')
db=client.rawData
collection=db.CostPower
try:
    collection.drop()
    collection.insert_many(costPower,ordered=False)
except:
    print("some data has same _id")
    
print("xxx")
client.close()

['ipeenWebsite', 'bigStyleCount', 'ipeenInfo', 'smallStyleCount', 'addressCoordinate', 'HRdata104', 'CostPower', 'Nhuman']
xxx


# humanDensity

In [28]:
import requests
import json
import pymongo
import re
# res=requests.get("https://quality.data.gov.tw/dq_download_json.php?nid=32973&md5_url=52629fae1f1fed4ef81fcf706878dd80")
# x=json.loads(res.text)

res=requests.get("http://data.moi.gov.tw/MoiOD/System/DownloadFile.aspx?DATA=3AF951B6-534D-46CB-8276-E158B6D9EE41")
datas=res.text.split("\r\n")
titles=datas[0].split(",")
x=[{title:value for title,value in zip(titles,data.split(","))} for data in datas[2:]]

data=[{"add":add['site_id'].replace("　","").replace("\U000fffb5","五")+add['village'].replace(" ","").replace("\U000fffb5","五").replace("?","館"),
       'Nhuman':add['people_total']}for add in x[1:] if "site_id" in add]
for i in data:
    try:
        i["bigadd"]=re.findall("(..[市|縣])(\w\w?\w?[區|市|鎮|鄉])",
                               i["add"])[0][0].replace("臺","台")
    except:
        i['bigadd']=0
    try:
        i["smalladd"]=re.findall("(..[市|縣])(\w\w?\w?[區|市|鎮|鄉])",
                           i["add"])[0][1]
    except:
        i["smalladd"]=0

client=pymongo.mongo_client.MongoClient("localhost",27017,username='j122085',password='850605')
# print(client.database_names())
db=client.rawData
print(db.collection_names())
collection=db.addressCoordinate
Coordinate={add['add']:{"lat":add['lat'],'lng':add['lng']} for add in list(collection.find({}))}
client.close()

for add in data:
    try:
        add['lat']=Coordinate[add['add']]['lat']
        add['lng']=Coordinate[add['add']]['lng']
        add["_id"]=add.pop('add')
    except:
#         print(add['add'])
        pass
Nhuman=[add for add in data if 'lat' in add]
client=pymongo.mongo_client.MongoClient("localhost",27017,username='j122085',password='850605')
db=client.rawData
collection=db.Nhuman
try:
    collection.drop()
    collection.insert_many(Nhuman,ordered=False)
except:
    print("some data has same _id")
client.close()

['ipeenWebsite', 'bigStyleCount', 'ipeenInfo', 'smallStyleCount', 'addressCoordinate', 'HRdata104', 'CostPower', 'Nhuman']


In [30]:
client=pymongo.mongo_client.MongoClient("localhost",27017,username='j122085',password='850605')
db=client.rawData
collection=db.HRdata104
x=list(collection.find({}))

In [33]:
set([i['bigadd'] for i in x])

{'南投縣',
 '台中市',
 '台北市',
 '台南市',
 '嘉義市',
 '嘉義縣',
 '基隆市',
 '宜蘭縣',
 '彰化縣',
 '新北市',
 '新竹市',
 '新竹縣',
 '桃園市',
 '苗栗縣',
 '雲林縣'}

In [24]:
data[0]

{'Nhuman': '1567',
 '_id': '新北市板橋區流芳里',
 'bigadd': '新北市',
 'lat': 25.0063524,
 'lng': 121.4568661,
 'smalladd': '板橋區'}

In [35]:
client=pymongo.mongo_client.MongoClient("localhost",27017,username='j122085',password='850605')
db=client.rawData
collection=db.Nhuman
len(list(collection.find({})))

7659

In [134]:
client=pymongo.mongo_client.MongoClient("172.20.26.39",27017,username='j122085',password='850605')
print(client.database_names())
db=client.rawData
print(db.collection_names())
collection=db.addressCoordinate
Coordinate=list(collection.find({}))
for i in Coordinate:
    del i["_id"]
import json
with open('../data/Coordinate.json','w') as f:
    json.dump(Coordinate,f)

['admin', 'local', 'rawData']
['ipeenWebsite', 'Nhumnan', 'ipeenInfo', 'addressCoordinate', 'HRdata104', 'CostPower', 'Nhuman']


# 將區里經緯度資訊放進mongo內

In [1]:
import json
import pymongo
with open('../data/Coordinate.json') as f:
    coordinate=json.load(f)
    
client=pymongo.mongo_client.MongoClient("localhost",27017,username='j122085',password='850605')
db=client.rawData
collection=db.addressCoordinate
collection.insert_many(coordinate)

In [41]:
res=requests.get("http://data.moi.gov.tw/MoiOD/System/DownloadFile.aspx?DATA=3AF951B6-534D-46CB-8276-E158B6D9EE41")
datas=res.text.split("\r\n")
titles=datas[0].split(",")
x=[{title:value for title,value in zip(titles,data.split(","))} for data in datas[2:]]

data=[{"add":add['site_id'].replace("　","").replace("\U000fffb5","五")+add['village'].replace(" ","").replace("\U000fffb5","五").replace("?","館"),'Nhuman':add['people_total']}for add in x[1:]]

KeyError: 'site_id'